In all versions 4.x the narrative gets cleaned as before and products are cleaned twice times. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time , the components that almost never occur are removed. This run uses 20 different principle components. 

__Initialing the data__

In [1]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_first_cut_scarce_elimination.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 0


__Initializing the bag of words__

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [3]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=20)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(242494, 20)


In [4]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.26616408 -3.37081876  0.01885175 ...  0.16645132 -0.69728048
  -0.73026482]
 [ 3.96354958 -1.09940075  0.72588229 ...  0.38966973 -0.67378058
  -0.72242037]
 [14.4719187   7.74607887 -1.5050687  ...  4.23931884  7.2940579
  -2.82344571]
 ...
 [ 0.72788142  0.60665734 -0.70433433 ... -0.01592422  0.12462253
   0.08442982]
 [ 5.77501709  4.62783088 -5.63719091 ... -1.56446705  0.5018494
   0.81687103]
 [ 6.52529359  1.6021713   0.53302629 ...  0.6842801   0.5692837
   0.59640081]]
                0         1         2         3         4         5         6  \
0        4.266164 -3.370819  0.018852  1.006848  0.620661 -0.214125 -0.556896   
1        3.963550 -1.099401  0.725882 -0.560909  0.338411  2.805625 -0.221218   
2       14.471919  7.746079 -1.505069  1.585133  0.697662 -4.421009 -2.918772   
3        2.018344  0.900111  2.367294 -0.566768 -0.509549  1.863570  0.255314   
4        4.969348 -4.724672 -1.145415 -3.919261  3.566865  0.822860 -1.410481   
...           ...       .

__One hot encoding of extra columns__

In [5]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(242494, 20)
(242494, 146)
(242494, 166)


__Split into train and test__

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(181870, 166)
(60624, 166)
(181870,)
(60624,)


__Application of linear SVM__

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


                                                    precision    recall  f1-score   support

                           Bank account or service       1.00      1.00      1.00      3777
                       Checking or savings account       0.99      1.00      1.00      4740
                                     Consumer Loan       0.95      0.78      0.86      2904
                                       Credit card       1.00      1.00      1.00      4743
                       Credit card or prepaid card       0.99      0.95      0.97      8168
                                  Credit reporting       1.00      1.00      1.00      7922
                                   Debt collection       1.00      1.00      1.00      9880
Money transfer, virtual currency, or money service       0.99      0.99      0.99      1935
                                          Mortgage       0.99      1.00      1.00      7556
         Payday loan, title loan, or personal loan       0.82      0.97      0.